In [ ]:
import cv2
import numpy as np

def detect_face(image):
    """
    Simulates the detection of a face in an image.

    Args:
        image (np.ndarray): The input image.

    Returns:
        tuple or None: A tuple (x, y, w, h) for the bounding box, or None if no face is found.
    """
    if np.all(image == 0):
        return None

    h, w = image.shape[:2]
    face_w = int(w * 0.6)
    face_h = int(h * 0.7)
    face_x = (w - face_w) // 2
    face_y = (h - face_h) // 2

    if image.shape[1] == 200:
         return (face_x, face_y, 50, 50)

    return (face_x, face_y, face_w, face_h)

def assess_frame_quality(
    image,
    t_blur=100.0,
    t_contrast=50.0,
    t_resolution=100
):
    """
    Evaluates a single video frame based on face presence, resolution, sharpness, and contrast.

    Args:
        image (np.ndarray): The input image frame (in BGR format from OpenCV).
        t_blur (float): The sharpness threshold (minimum variance of Laplacian).
        t_contrast (float): The contrast threshold (minimum pixel intensity range).
        t_resolution (int): The minimum required face width and height in pixels.

    Returns:
        bool: True if the frame is high quality, False otherwise.
        str: A message indicating the reason for the decision.
    """
    face_bbox = detect_face(image)
    if face_bbox is None:
        return False, "No face detected"

    x, y, w, h = face_bbox

    if w < t_resolution or h < t_resolution:
        return False, f"Resolution check failed: {w}x{h} is below {t_resolution}px"

    face_roi = image[y:y+h, x:x+w]
    gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

    laplacian_var = cv2.Laplacian(gray_face, cv2.CV_64F).var()
    if laplacian_var < t_blur:
        return False, f"Blur check failed: {laplacian_var:.2f} is below threshold {t_blur}"

    p10, p90 = np.percentile(gray_face, (10, 90))
    contrast_range = p90 - p10
    if contrast_range < t_contrast:
        return False, f"Contrast check failed: {contrast_range:.2f} is below threshold {t_contrast}"

    return True, "Frame passed all quality checks"

if __name__ == "__main__":
    BLUR_THRESHOLD = 100.0
    CONTRAST_THRESHOLD = 50.0
    RESOLUTION_THRESHOLD = 100

    high_quality_img = np.random.randint(0, 256, (480, 640, 3), dtype=np.uint8)
    high_quality_img = cv2.normalize(high_quality_img, None, 0, 255, cv2.NORM_MINMAX)

    blurry_img = cv2.GaussianBlur(high_quality_img, (25, 25), 0)

    low_contrast_img = np.full((480, 640, 3), 128, dtype=np.uint8)
    low_contrast_img = low_contrast_img + np.random.randint(-20, 20, (480, 640, 3), dtype=np.uint8)
    low_contrast_img = np.clip(low_contrast_img, 0, 255).astype(np.uint8)

    low_res_img = np.random.randint(0, 256, (240, 200, 3), dtype=np.uint8)

    no_face_img = np.zeros((480, 640, 3), dtype=np.uint8)

    test_cases = {
        "High Quality": high_quality_img,
        "Blurry": blurry_img,
        "Low Contrast": low_contrast_img,
        "Low Resolution": low_res_img,
        "No Face": no_face_img
    }

    print("--- Running Frame Quality Assessment ---")
    for name, img in test_cases.items():
        is_high_quality, reason = assess_frame_quality(
            img,
            t_blur=BLUR_THRESHOLD,
            t_contrast=CONTRAST_THRESHOLD,
            t_resolution=RESOLUTION_THRESHOLD
        )
        print(f"Image: '{name}' -> Quality Passed: {is_high_quality} ({reason})")
